In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from simulator.simulator import Simulator


def keypoints_to_colors(x, y, width, height):
    # Normalize coordinates to range [0, 1]
    x_norm = x / width
    y_norm = y / height

    # Compute hue based on position
    hue = (x_norm + y_norm) / 2  # Simple gradient approach
    # Alternative: hue = (np.arctan2(y_norm - 0.5, x_norm - 0.5) / (2 * np.pi)) % 1

    # Set saturation and value to 1
    hsv = np.stack([hue * 179, np.ones_like(hue) * 255, np.ones_like(hue) * 255], axis=-1).astype(np.uint8)

    # Convert HSV to RGB
    return cv2.cvtColor(hsv[np.newaxis, :, :], cv2.COLOR_HSV2RGB)[0]

In [ ]:
simulator = Simulator(
    blend_file='/home/clementin/Dev/blender-construction/construction_allimages.blend',
    points_density=100.0,
    verbose=True
)

In [ ]:
masks, uvs = [], []

for _ in range(16):
    rgb, depth, labels = simulator.render()
    _, _, mask, uv = simulator.compute_point_cloud()
    masks.append(mask)
    uvs.append(uv)
    print(simulator.scene.frame_current)
    plt.imshow(rgb)
    plt.show()
    simulator.step_frame()

uvs = np.stack(uvs)
masks = np.stack(masks)

valid = np.all(masks, axis=0)
uvs = uvs[:, :, valid]

# Blocks of 32x32
_, valid = np.unique(np.int64(uvs[0] / 32), axis=1, return_index=True)
uvs = uvs[:, :, valid]

h, w, _ = rgb.shape
colors = keypoints_to_colors(*uvs[0], w, h) / 255

In [ ]:
plt.imshow(rgb)

for uv, c in zip(uvs.T, colors):
    plt.plot(*uv, c=c, lw=1, alpha=0.3)

plt.scatter(*uvs[-1], c=colors, s=2, alpha=0.7)

In [ ]:
#plt.figure(figsize=(20, 20))
plt.imshow(cv2.imread('all_end.png')[:, :, ::-1])

for uv, c in zip(uvs.T, colors):
    plt.plot(*(uv * 1), c=c, lw=1, alpha=0.5)

plt.scatter(*(uvs[-1] * 1), c=colors, s=2)
plt.axis('off')
plt.savefig('matching.png', bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
depth[depth > 98] = 0

In [ ]:
plt.imshow(np.log(0.5 * depth + 40), cmap='RdYlGn')
plt.axis('off')
plt.savefig('depth.png', bbox_inches='tight', pad_inches=0, dpi=300)

In [ ]:
label_colors = {
    'Building': np.array([90, 90, 90], dtype=np.uint8),  # Neutral gray
    'Scaffolding': np.array([230, 180, 190], dtype=np.uint8),  # Soft red
    'Plastic barrel': np.array([0, 80, 200], dtype=np.uint8),  # Bright blue
    'CardboardBox': np.array([210, 180, 140], dtype=np.uint8),  # Light brown
    'Wheelbarrow': np.array([150, 40, 50], dtype=np.uint8),  # Rusty brown
    'Boulder': np.array([170, 160, 150], dtype=np.uint8),  # Rock gray
    'Grass': np.array([60, 180, 75], dtype=np.uint8),  # Vibrant green
    'Crane': np.array([255, 165, 0], dtype=np.uint8),  # Construction orange
    'Container': np.array([200, 0, 0], dtype=np.uint8),  # Bold red
    'Steel beam': np.array([128, 128, 128], dtype=np.uint8),  # Steel gray
    'Acrow Prop': np.array([140, 160, 60], dtype=np.uint8),  # Yellow-green for visibility
    'PVC': np.array([50, 50, 50], dtype=np.uint8),  # Almost black
    'Palette': np.array([130, 140, 90], dtype=np.uint8),  # Warm wood tone
    'Terrain': np.array([150, 110, 90], dtype=np.uint8),  # Reddish-brown for contrast
    'Bulldozer': np.array([255, 200, 0], dtype=np.uint8),  # High-visibility yellow
    'WoodenBox': np.array([190, 120, 60], dtype=np.uint8),  # Darker wood tone
    'Toilet': np.array([80, 160, 220], dtype=np.uint8),  # Light blue
    'Fence': np.array([30, 150, 120], dtype=np.uint8),  # Teal green
    'Wooden planks': np.array([175, 120, 60], dtype=np.uint8),  # Golden brown  
    'Generator engine': np.array([40, 60, 90], dtype=np.uint8), # Dark navy blue  
    'Truck': np.array([0, 130, 200], dtype=np.uint8),           # Strong cyan blue  
    'Concrete mixer': np.array([240, 130, 40], dtype=np.uint8), # Bright orange  
    'Cement bag': np.array([190, 180, 160], dtype=np.uint8)     # Dusty gray-beige
}

In [ ]:
colors = np.zeros_like(rgb)
for l in np.unique(labels):
    if l >= 0:
        label_color = label_colors[simulator.labels[l]] / 255
        colors[labels == l] = label_color

In [ ]:
plt.imshow(colors)
plt.axis('off')
plt.savefig('semantic.png', bbox_inches='tight', pad_inches=0, dpi=300)